CRIAÇÃO E EXTRAÇÃO DOS ARQUIVOS COM ZIPFILE

In [1]:
import requests 

url="https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip"

r=requests.get(url, verify=False)


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
with open("r.zip","wb") as f:
    f.write(r.content)
    
import zipfile 

with zipfile.ZipFile("r.zip","r") as zip_ref:
    zip_ref.extractall("./data")

SPARKSESSION PARA TRATAMENTO DE DADOS

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lower

spark = SparkSession \
        .builder \
        .config("spark.sql.repl.eagerEval.enabled", "True") \
        .config("spark.sql.repl.eagerEval.maxNumRows", "10") \
        .getOrCreate()

df = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV", sep=';',header=True, inferSchema=True)


TRANSFORMANDO LETRAS PARA MINUSCULAS

In [4]:
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

CONVERTENDO ARQUIVO "CSV" PARA "PARQUET"

In [5]:
df.write.parquet("MICRODADOS_IES_CONSULTA.parquet")

parquet_df = spark.read.parquet("MICRODADOS_IES_CONSULTA.parquet")

parquet_df.createOrReplaceTempView("parquet_df")


CONSULTAS

1º questão- Olhando para o estado de Minas Gerais, quantos municípios têm informações presentes na base de dados?

In [6]:
spark.sql("SELECT COUNT(*) as Qtd_Municipios_Presentes FROM \
            (SELECT DISTINCT no_municipio_ies \
             FROM parquet_df \
             WHERE no_uf_ies = 'Minas Gerais') \
          ")

Qtd_Municipios_Presentes
104


2º Questão- Quantos professores doutores existem em cada cidade de Minas Gerais presente na base de dados ?

In [7]:
spark.sql("""SELECT no_municipio_ies as municipio, sum(qt_doc_ex_dout) as Qtd_Doutores \
             from parquet_df \
             WHERE no_uf_ies = 'Minas Gerais' \
             group by no_municipio_ies \
             order by Qtd_Doutores DESC""")

municipio,Qtd_Doutores
Belo Horizonte,6615
Uberl�ndia,1939
Juiz de Fora,1915
Vi�osa,1185
Montes Claros,925
Uberaba,891
Lavras,812
S�o Jo�o del Rei,753
Ouro Preto,746
Diamantina,673


3º Questão- Qual a quantidade de docentes com deficiência no estado do Paraná?

In [8]:
spark.sql("SELECT SUM(qt_doc_ex_com_deficiencia) as Qtd_docentes_deficientes \
          from parquet_df \
          WHERE sg_uf_ies= 'PR' ")

Qtd_docentes_deficientes
160


FIM DO DESAFIO... POR ENQUANTO... ;)